In [1]:
import pandas as pd
import numpy as np
import openmatrix as omx
import os, sys, glob

from dbfread import DBF

In [2]:
os.chdir(r"C:\Users\vyadav\Cambridge Systematics\PROJ 210071 BART Link21 TDLU Modeling - Documents\Task 2 - Model Dev\2.3 - Model Construction\Nonres\HSR trips")

In [3]:
out_dir = r"C:\Users\vyadav\Cambridge Systematics\PROJ 210071 BART Link21 TDLU Modeling - Documents\Task 2 - Model Dev\2.3 - Model Construction\Nonres\HSR trips"

In [4]:
def df2dbf(df, dbf_path, my_specs=None):
    '''


    Convert a pandas.DataFrame into a dbf.
    __author__  = "Dani Arribas-Bel <darribas@asu.edu> "
    ...
    Arguments
    ---------
    df          : DataFrame
                  Pandas dataframe object to be entirely written out to a dbf
    dbf_path    : str
                  Path to the output dbf. It is also returned by the function
    my_specs    : list
                  List with the field_specs to use for each column.
                  Defaults to None and applies the following scheme:
                    * int: ('N', 14, 0)
                    * float: ('N', 14, 14)
                    * str: ('C', 14, 0)

    from: https://github.com/GeoDaSandbox/sandbox/blob/master/pyGDsandbox/dataIO.py

    Copyright (c) 2007-2011, GeoDa Center for Geospatial Analysis and Computation
    All rights reserved.

    Redistribution and use in source and binary forms, with or without
    modification, are permitted provided that the following conditions are met:

    * Redistributions of source code must retain the above copyright notice, this
      list of conditions and the following disclaimer.

    * Redistributions in binary form must reproduce the above copyright
      notice, this list of conditions and the following disclaimer in the
      documentation and/or other materials provided with the distribution.

    * Neither the name of the GeoDa Center for Geospatial Analysis and Computation
      nor the names of its contributors may be used to endorse or promote products
      derived from this software without specific prior written permission.

    THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND
    CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES,
    INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF
    MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
    DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR
    CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
    SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT
    LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF
    USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
    ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT
    LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN
    ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
    POSSIBILITY OF SUCH DAMAGE.
    '''

    import libpysal.io as ps
    import numpy as np
    if my_specs:
        specs = my_specs
    else:
        type2spec = {int: ('N', 20, 0),
                     np.int64: ('N', 20, 0),
                     float: ('N', 36, 15),
                     np.float64: ('N', 36, 15),
                     str: ('C', 14, 0)
                     }
        types = [type(df[i].iloc[0]) for i in df.columns]
        specs = [type2spec[t] for t in types]
    db = ps.open(dbf_path, 'w')
    db.header = list(df.columns)
    db.field_spec = specs
    for i, row in df.T.iteritems():
        db.write(row)
    db.close()
    return dbf_path

In [5]:
# read and validate crosswalk
xwalk = pd.read_csv(r"Disagg TM1.5\TM1_to_Link21_HSRZones.csv")

# every TM1.5 and TM2 zone is accounted for
# factors sum up to 1 for every TM1.5 zone

missing_zone_IP = set(range(1, max(xwalk.IPZONE) + 1)) - set(xwalk.IPZONE)
missing_zone_OP = set(range(1, max(xwalk.OPZONE) + 1)) - set(xwalk.OPZONE)
assert len(missing_zone_IP) == 0, f'missing TM1.5 zones, {missing_zone_IP}'
assert len(missing_zone_OP) == 0, f'missing TM2 zones, {missing_zone_OP}'

assert (xwalk.groupby(['IPZONE'])['FACTOR'].sum().round(5).nunique() == 1 and
        xwalk.groupby(['IPZONE'])['FACTOR'].sum().round(5).max() == 1), 'factors do not add up to one for every TM1.5 zone'

In [6]:
max(xwalk.IPZONE), max(xwalk.OPZONE)

(1475, 3353)

In [7]:
#segment_suffixes = ["LowInc", "MedInc", "HighInc", "XHighInc"]
#shares = [0.28, 0.24, 0.21, 0.27]
in_table_regex = "Disagg TM1.5/tripsHsr{period}_{year}.dbf"
out_omx_regex = os.path.join(out_dir, r"disagg/{year}/tripsHsr{period}_{year}.dbf")
modes = ["DA","SR2","TRANSIT","WALK"] # from tm2py model_config.toml

In [9]:
%%time
for year in ['2040', '2050']:
    
    if not os.path.exists(f'disagg/{year}'):
        os.mkdir(f'disagg/{year}')

    for time_period in ['EA','AM','MD','PM','EV']:
        df = pd.DataFrame(
            DBF(in_table_regex.format(year = year, period = time_period), load = True).records)
        
        print(df.columns)
        for cols in df.columns:
            print(f'{cols} sum =', df[cols].sum())

        if 'S2' in df.columns:
            df.rename(columns = {'S2':'SR2'}, inplace = True)
        if 'S3' in df.columns:
            df.rename(columns = {'S3':'SR3'}, inplace = True)
        
        
        df_w_xwalk = df.merge(
                xwalk, left_on = ['ORIG'], right_on = ['IPZONE'], how = 'left').merge(
                xwalk, left_on = ['DEST'], right_on = ['IPZONE'], how = 'left', suffixes = ['_ORIG','_DEST'])

        #OD_full_index = pd.MultiIndex.from_product([range(1,max(xwalk.OPZONE) + 1), range(1,max(xwalk.OPZONE) + 1)])

        for mode in modes:
            #print(mode)
            df_w_xwalk[f'{mode}_new'] = df_w_xwalk[mode] * df_w_xwalk.FACTOR_ORIG * df_w_xwalk.FACTOR_DEST
            print(f'{mode} - ', df_w_xwalk[f'{mode}'].sum(), df_w_xwalk[f'{mode}_new'].sum())
            
            #if df_w_xwalk[f'{mode}'].sum() != df_w_xwalk[f'{mode}_new'].sum():
            #    break
                
        df_w_xwalk = df_w_xwalk[['OPZONE_ORIG', 'OPZONE_DEST', 'DA_new', 'SR2_new', 'TRANSIT_new', 'WALK_new']]
        
        df_w_xwalk = df_w_xwalk.rename(columns={
            'OPZONE_ORIG' : 'ORIG',
            'OPZONE_DEST' : 'DEST',
            'DA_new' : 'DA',
            'SR2_new' : 'SR2',
            'TRANSIT_new' : 'TRANSIT',
            'WALK_new' : 'WALK'
        })
        
        print(f'writing DBF file for {year} and {time_period}')
        
        df_w_xwalk = df_w_xwalk.groupby(['ORIG', 'DEST'])['DA', 'SR2', 'TRANSIT', 'WALK'].sum().reset_index()
        
        df2dbf(df_w_xwalk, out_omx_regex.format(year = year, period=time_period))

Index(['ORIG', 'DEST', 'DA', 'SR2', 'TRANSIT', 'WALK'], dtype='object')
ORIG sum = 1599090273
DEST sum = 1599090273
DA sum = 0.0
SR2 sum = 0.0
TRANSIT sum = 0.0
WALK sum = 0.0
DA -  0.0 0.0
SR2 -  0.0 0.0
TRANSIT -  0.0 0.0
WALK -  0.0 0.0
writing DBF file for 2040 and EA


<timed exec>:47: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Users\vyadav\Anaconda3\envs\runway37\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


Index(['ORIG', 'DEST', 'DA', 'SR2', 'TRANSIT', 'WALK'], dtype='object')
ORIG sum = 1599090273
DEST sum = 1599090273
DA sum = 4334.540000000001
SR2 sum = 3284.6
TRANSIT sum = 7420.280000000004
WALK sum = 840.9699999999998
DA -  24959.65999999999 4334.540000062003
SR2 -  17909.830000000016 3284.600000039931
TRANSIT -  35832.88 7420.280000093321
WALK -  5520.700000000002 840.9699999635798
writing DBF file for 2040 and AM


<timed exec>:47: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


Index(['ORIG', 'DEST', 'DA', 'SR2', 'TRANSIT', 'WALK'], dtype='object')
ORIG sum = 1599090273
DEST sum = 1599090273
DA sum = 6278.749999999994
SR2 sum = 7902.329999999997
TRANSIT sum = 4150.349999999999
WALK sum = 4339.270000000003
DA -  32286.02999999999 6278.750000048538
SR2 -  40621.15999999997 7902.330000056912
TRANSIT -  18900.44999999996 4150.350000029249
WALK -  28864.62999999999 4339.269999896821
writing DBF file for 2040 and MD


...

Index(['ORIG', 'DEST', 'DA', 'SR2', 'TRANSIT', 'WALK'], dtype='object')
ORIG sum = 1599090273
DEST sum = 1599090273
DA sum = 4334.540000000001
SR2 sum = 3284.6
TRANSIT sum = 7420.280000000004
WALK sum = 786.3099999999998
DA -  24959.65999999999 4334.540000062003
SR2 -  17909.830000000016 3284.600000039931
TRANSIT -  35832.88 7420.280000093321
WALK -  5506.4 786.3099999697099
writing DBF file for 2050 and PM


<timed exec>:47: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


Index(['ORIG', 'DEST', 'DA', 'SR2', 'TRANSIT', 'WALK'], dtype='object')
ORIG sum = 1599090273
DEST sum = 1599090273
DA sum = 6278.749999999994
SR2 sum = 7902.329999999997
TRANSIT sum = 4150.349999999999
WALK sum = 4339.270000000003
DA -  32286.02999999999 6278.750000048538
SR2 -  40621.15999999997 7902.330000056912
TRANSIT -  18900.44999999996 4150.350000029249
WALK -  28864.62999999999 4339.269999896821
writing DBF file for 2050 and EV


<timed exec>:47: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


CPU times: total: 1h 23min 4s
Wall time: 1h 23min 29s


In [19]:
in_file = r'C:\Users\vyadav\Cambridge Systematics\PROJ 210071 BART Link21 TDLU Modeling - Documents\Task 2 - Model Dev\2.3 - Model Construction\Nonres\HSR trips\disagg\2040'

In [20]:
file = os.path.join(in_file, 'tripsHsrEA_2040.dbf')

In [21]:
df = pd.DataFrame(
            DBF(file, load = True).records)

In [22]:
df

,ORIG,DEST,DA,SR2,TRANSIT,WALK
0,83,83,0.0,0.0,0.0,0.0
1,83,62,0.0,0.0,0.0,0.0
2,83,85,0.0,0.0,0.0,0.0
3,83,63,0.0,0.0,0.0,0.0
4,83,86,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
11269444,3351,3347,0.0,0.0,0.0,0.0
11269445,3351,3348,0.0,0.0,0.0,0.0
11269446,3351,3349,0.0,0.0,0.0,0.0
11269447,3351,3350,0.0,0.0,0.0,0.0


In [25]:
test = df.groupby(['ORIG', 'DEST'])['DA', 'SR2', 'TRANSIT', 'WALK'].sum().reset_index()

C:\Users\vyadav\AppData\Local\Temp\ipykernel_17264\2280661467.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  test = df.groupby(['ORIG', 'DEST'])['DA', 'SR2', 'TRANSIT', 'WALK'].sum().reset_index()


In [26]:
test

,ORIG,DEST,DA,SR2,TRANSIT,WALK
0,1,1,0.0,0.0,0.0,0.0
1,1,2,0.0,0.0,0.0,0.0
2,1,3,0.0,0.0,0.0,0.0
3,1,4,0.0,0.0,0.0,0.0
4,1,5,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
11229196,3351,3347,0.0,0.0,0.0,0.0
11229197,3351,3348,0.0,0.0,0.0,0.0
11229198,3351,3349,0.0,0.0,0.0,0.0
11229199,3351,3350,0.0,0.0,0.0,0.0
